# VacationPy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#creating path to output data csv
weatherInfoFile = "../output_data/weatherInfo.csv"

#reading in csv
uglyWeather = pd.read_csv(weatherInfoFile)

#deletes the first column named unnamed, which counted from 0-the end of the datset for no reason. 
weather = uglyWeather.drop(columns=["Unnamed: 0"])
weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed
0,New,75,LR,1579938433,88,6.65,-10.59,71.60,4.70
1,Punta,51,PH,1579938433,89,8.99,125.34,80.89,1.16
2,Grand,90,US,1579938433,100,38.88,-109.50,32.00,1.03
3,Paraiso,96,CR,1579938310,77,9.84,-83.87,68.00,6.93
4,Eyemouth,100,GB,1579938434,87,55.87,-2.09,45.00,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Creating coordinates and humidity lists which will be added to the heatmap
coordinates = weather[['Lat.','Lng.']]

humidity = weather['Humidity']

#Map center will focus the figure below on Kriti, in the mediterranean sea to give the map a nice focal point
mapCenter = (35.2401, 24.8093)

# Plot Heatmap, setting center of map, and zoom level to 2 to get rid of multiple maps in same fig
fig = gmaps.figure(center=mapCenter,zoom_level=1.0)

#heat map with weight set to humidity to showcase which cities are the most humid 
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Drop country, date,lat, and long values to focus on weather data and city names 
updated_df = pd.DataFrame(weather, columns = ["City","Cloudiness","Max Temp", "Wind Speed"])

#Now I get to be picky and set the values of the weather that I enjoy most
#Temperature should be 65-80 F
#Wind should be minimal, no more than 5 mph
#Clear sky is better than cloudy sky, I set the max cloudiness value to 5%
#Drop all rows that do not fit this criteria

idealCloud = updated_df["Cloudiness"] <=5
idealTemp = ((updated_df["Max Temp"] <=80) & (updated_df["Max Temp"] > 65))
idealWind = updated_df["Wind Speed"] < 5

idealCities = updated_df[idealCloud & idealTemp & idealWind]
narrowed_city_df = pd.DataFrame(idealCities)
narrowed_city_df

,City,Cloudiness,Max Temp,Wind Speed
33,Hermanus,0,66.00,3.00
119,Nanakuli,1,75.20,4.70
225,Mattru,0,79.29,2.26
252,Marzuq,2,79.77,4.63
289,Florianopolis,0,68.00,4.70
315,Balotra,0,76.71,3.56
323,Jawad,0,75.33,4.34
346,Luderitz,0,71.60,1.12
432,Panguna,5,71.33,0.45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#creating a dataframe which is strictly geographical with no weather information
hotel_df = pd.DataFrame(weather, columns = ["City" , "Country", "Lat.", "Lng."])

#creating empty column to hold hotel names later
hotel_df["Hotel Names"] = ''
hotel_df

,City,Country,Lat.,Lng.,Hotel Names
0,New,LR,6.65,-10.59,
1,Punta,PH,8.99,125.34,
2,Grand,US,38.88,-109.50,
3,Paraiso,CR,9.84,-83.87,
4,Eyemouth,GB,55.87,-2.09,
...,...,...,...,...,...
538,Mahabalipuram,IN,12.63,80.17,
539,Hanzhong,CN,33.07,107.03,
540,Yangambi,CD,0.81,24.43,
541,Sept-Iles,CA,50.20,-66.38,


# Calling the Google API

In [6]:
target_coordinates = f"{hotel_df['Lat.'][0]},{hotel_df['Lng.'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

#setting parameters to make the call to the API
#example url from google for reference
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

#requesting information with json 
response = requests.get(url, params=params)

In [ ]:
hotel_name = []

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat."]
    lng = row["Lng."]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

#hotel_df['Hotel Name']= hotel_name
#hotel_df.dropna()
#hotel_df.to_csv('hotel_export.csv')

In [ ]:
hotel_name

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat.", "Lng."]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map